In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Standard procedures

Some boilerplate:

In [ ]:
dataset_file = ''
df = pd.read_csv(dataset_file, names=['cols'], index_col=0)

print("The dataset size is {}".format(df.shape))
df.head(5)

# Preprocessing

Dropping columns:

In [ ]:
df = df.drop(columns=['cols'])

N.B: You should drop columns with a very similar distribution to other coloumns.

Deleting nulls (na) values:

In [ ]:
print("There are {} columns with missing values".format(df.isna().sum().sum()))
df = df.dropna()
print("There are {} columns with missing values".format(df.isna().sum().sum()))

OneHotEncoding (it is preferred when dealing with non-ordinal attributes):

MinMax scaling, especially useful in clustering

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_processed = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Plots

In [ ]:
import seaborn as sns
sns.pairplot(df)

# Classification

# Clustering

### Agglomerative Clustering

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score

k_range = list(range(2,11)) # set the range of k values to test 

parameters = [{'n_clusters': k_range
                    , 'linkage' : ['ward', 'complete', 'average', 'single']}]
pg = list(ParameterGrid(parameters))
result_ac = []

for i in range(len(pg)):
    ac = AgglomerativeClustering(**(pg[i]))
    y_ac = ac.fit_predict(df)
    result_ac.append([pg[i]['linkage'],pg[i]['n_clusters'],silhouette_score(df, y_ac)])


# Dataframe with the results
df_result_ac = pd.DataFrame(data = result_ac, columns=['linkage','n_clusters','silhouette_score'])
df_result_ac.sort_values(by='silhouette_score', ascending=False).head(5)

### DBSCAN method

### KMeans

In [ ]:
from sklearn.cluster import KMeans 
from sklearn.metrics import silhouette_score

k_range = list(range(2,11)) # set the range of k values to test 
results = []

for k_ in k_range : 
    estimator = KMeans(n_clusters=k_)
    y_pred = estimator.fit_predict(df)
    results.append([k_, silhouette_score(df, y_pred), estimator.inertia_])

results = pd.DataFrame(data=results, columns=['n_clusters', 'sil_score', 'inertia'])
results

Plot inertias in KMeans, useful for elbow method

In [ ]:
fig, ax = plt.subplots()
ax.plot(k_range, results['inertia'], color='red')
ax.set_xlabel('Number of clusters')
ax.set_ylabel('Inertias', color='red')

ax2 = ax.twinx()
ax2.plot(k_range, results['sil_score'], color='blue')
ax2.set_ylabel('Silhouette scores', color='blue')
ax2.set_ylim(0, 1)

plt.show()

# Association rules